# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
print(song_files[0])

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [6]:
filepath = song_files[0]

In [7]:
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [8]:
df = pd.read_json(filepath, lines=True) 
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [22]:
selected_columns = df[["song_id", "title", "artist_id", "year", "duration"]]
#row_to_insert = selected_columns.iloc[0]
row_to_insert = selected_columns.values[0]
#print(row_to_insert.shape)
#print(selected_columns)
print(row_to_insert)

['SOMZWCG12A8C13C480' "I Didn't Mean To" 'ARD7TVE1187B99BFB1' 0 218.93179]


In [23]:
song_data = row_to_insert.tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [24]:
# The below is not needed if we resign from REFERENCE to foreign key
# cur.execute("ALTER TABLE songs DROP CONSTRAINT songs_artist_id_fkey")

In [25]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [26]:
selected_columns = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
row_to_insert = selected_columns.values[0]
#print(row_to_insert.shape)
print(row_to_insert)

['ARD7TVE1187B99BFB1' 'Casual' 'California - LA' nan nan]


In [27]:
artist_data = row_to_insert.tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [28]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [5]:
log_files = get_files("data/log_data")

In [6]:
print(log_files[0])

/home/workspace/data/log_data/2018/11/2018-11-30-events.json


In [7]:
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-30-events.json


In [8]:
df = pd.read_json(filepath, lines=True) 
#df.head()
df.sample(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
137,Shania Twain,Logged In,Mohammad,M,5,Rodriguez,250.51383,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,1075,Nah!,200,1543566117796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
102,3OH!3,Logged In,Chloe,F,20,Cuevas,192.52200,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,1079,My First Kiss (Feat. Ke$ha) [Album Version],200,1543558403796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
329,Drop Dead_ Gorgeous,Logged In,Rylan,M,29,George,156.70812,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Dressed For Friend Requests (Album Version),200,1543596546796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
165,Nancy Ajram,Logged In,Matthew,M,16,Jones,221.67465,paid,"Janesville-Beloit, WI",PUT,NextSong,1.541063e+12,998,Meen Dah Elly Nseik,200,1543576803796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",36
286,The Cinematic Orchestra,Logged In,Rylan,M,12,George,622.26240,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Everyday,200,1543591920796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [9]:
df = df[df["page"]=="NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [10]:
df["timestamp"] = pd.to_datetime(df["ts"], unit='ms')
df["hour"] = df["timestamp"].dt.hour
df["day"] = df["timestamp"].dt.day
df["week"] = df["timestamp"].dt.week
df["month"] = df["timestamp"].dt.month
df["year"] = df["timestamp"].dt.year
df["weekday"] = df["timestamp"].dt.weekday_name
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,timestamp,hour,day,week,month,year,weekday
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,...,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91,2018-11-30 00:22:07.796,0,30,48,11,2018,Friday
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:08:41.796,1,30,48,11,2018,Friday
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:12:48.796,1,30,48,11,2018,Friday
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:17:05.796,1,30,48,11,2018,Friday
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:20:56.796,1,30,48,11,2018,Friday


In [35]:
#time_data = pd.concat([t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday], axis=1)
time_data = pd.concat([df["timestamp"], df["hour"], df["day"], df["week"], df["month"], df["year"], df["weekday"]], axis=1)

column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

time_data.columns = column_labels

In [ ]:
print(time_data)

In [37]:
time_df = time_data.drop_duplicates(subset=None, keep='first', inplace=False)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,Friday
1,2018-11-30 01:08:41.796,1,30,48,11,2018,Friday
2,2018-11-30 01:12:48.796,1,30,48,11,2018,Friday
3,2018-11-30 01:17:05.796,1,30,48,11,2018,Friday
4,2018-11-30 01:20:56.796,1,30,48,11,2018,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [38]:
#cur.execute("DELETE FROM time")

In [39]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [40]:
user_df_withduplicates = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df = user_df_withduplicates.drop_duplicates(subset=None, keep='first', inplace=False)
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
23,86,Aiden,Hess,M,free
30,24,Layla,Griffin,F,paid
40,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [41]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [42]:
# The below was a try to create a primary key, but is not needed since we used SERIAL PRIMARY KEY.

#df["key"] = df.reset_index().index
#df.head()

In [16]:
#cur.execute("DELETE FROM songplays")

In [ ]:
for index, row in df.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None  

    # insert songplay record
    selected_columns = df[["timestamp", "userId", "level", "sessionId", "location", "userAgent"]]
    row_to_insert = selected_columns.values[0]
    songplay_data = row_to_insert.tolist()
    
    songplay_data.append(songid)
    songplay_data.append(artistid)
     
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.